In [1]:
import pandas as pd
import numpy as np
from pybnesian import hc, CLGNetworkType, SemiparametricBNType, SemiparametricBN, MMHC, RCoT, MutualInformation, KMutualInformation, OperatorSet, Score, OperatorPool, ChangeNodeTypeSet, ArcOperatorSet, HoldoutLikelihood, BIC, BGe, CVLikelihood, CKDE, HCKDE, ValidatedLikelihood
#from drawdata import draw_scatter
import matplotlib.pyplot as plt

from bayesace.utils import *
from bayesace.algorithms.bayesace import BayesACE
from bayesace.algorithms.face import FACE

from sklearn.preprocessing import StandardScaler
import openml as oml

import time


In [2]:
df = oml.datasets.get_dataset(44091).get_data()[0]


/tmp/ipykernel_4048/1030542227.py:1: FutureWarning: Starting from Version 0.15 `download_data`, `download_qualities`, and `download_features_meta_data` will all be ``False`` instead of ``True`` by default to enable lazy loading. To disable this message until version 0.15 explicitly set `download_data`, `download_qualities`, and `download_features_meta_data` to a bool while calling `get_dataset`.
  df = oml.datasets.get_dataset(44091).get_data()[0]


In [3]:
df["class"] = df["quality"].astype('string').astype('category')
df = df.drop("quality", axis=1)

In [4]:
naive = SemiparametricBN(df.columns)

In [5]:
for i in [i for i in df.columns if i != "class"] :
    naive.add_arc("class", i)

In [6]:
df_train = df.head(2000)
df_test = df.tail(500)

In [7]:
test = MutualInformation(df_train, True)
#df = pd.read_csv("toy-3class.csv")
#df["class"] = df["z"].astype('category')
#df["z"] = df["z"].astype('category')
#df = df.drop("z", axis = 1 )

feature_columns = [i for i in df.columns if i != "class"]
df[feature_columns] = StandardScaler().fit_transform(df[feature_columns].values)



learned = hc(df_train, bn_type = CLGNetworkType(), operators = ["arcs"], score = "bic")
learned.fit(df_train)

est = MMHC()
learned_kde = MMHC().estimate(hypot_test = test, operators = OperatorPool([ChangeNodeTypeSet(),ArcOperatorSet()]), score = CVLikelihood(df_train), bn_type = SemiparametricBNType()) #, score = "cv-lik"
learned_kde = hc(df_train, start = naive, operators = ["node_type","arcs"], score = "holdout-lik", num_folds = 10)
learned_kde.fit(df_train)



In [347]:
for i in df.columns :
    print(i)
    print(learned_kde.cpd(i))
    print()

fixed acidity
[CKDE] P(fixed acidity | chlorides, residual sugar, alcohol, sulphates, total sulfur dioxide) = CKDE with 2000 instances

volatile acidity
[HCKDE] P(volatile acidity | chlorides, sulphates, pH, class, alcohol, density, total sulfur dioxide)
+-------+--------------------------------------------------------------------------------------------------------------------------+
|       |                   volatile acidity | chlorides, sulphates, pH, alcohol, density, total sulfur dioxide                    |
+-------+--------------------------------------------------------------------------------------------------------------------------+
| class |                                                                                                                          |
+-------+--------------------------------------------------------------------------------------------------------------------------+
| False | [CKDE] P(volatile acidity | chlorides, sulphates, pH, alcohol, density

In [348]:
len([i for i in learned_kde.arcs() if "class" in i])

5

In [71]:
n_exps = 20
df = df.sample(frac = 1)
df_train = df.head(2000)
for i in df_train.columns[:-2] :
    df_train = df_train[df_train[i] <4]
    df_train = df_train[df_train[i] >-4]
df_test = df.tail(500)
for i in df_test.columns[:-2] :
    df_test = df_test[df_test[i] <4]
    df_test = df_test[df_test[i] >-4]

labels = np.zeros(shape = (len(df_test.index)))
labels[df_test["class"] == "True"] = 1

sum_sum = []
accuracy = []
ll = []
num_arcs = []
class_arcs = []
times = []
for i in range(n_exps) :
    t0 = time.process_time()
    learned_kde = hc(df_train, start = naive, operators = ["node_type","arcs"], score = "validated-lik", num_folds = 10)
    #MMHC().estimate(hypot_test = test, operators = OperatorPool([ChangeNodeTypeSet(),ArcOperatorSet()]), score = CVLikelihood(df_train), bn_type = SemiparametricBNType())
    learned_kde.fit(df_train)
    times.append(time.process_time()-t0)
    saved = predict_class(df_test, learned_kde)
    accuracy.append(np.sum((labels-saved["True"])**2) / len(df_test.index))
    ll.append(learned_kde.slogl(df_train)/500)
    num_arcs.append(learned_kde.num_arcs())
    class_arcs.append(len([j for j in learned_kde.arcs() if "class" in j]))
    sum = 0
    for j in learned_kde.nodes()[:-2] :    
        if isinstance(learned_kde.cpd(j), CKDE) or isinstance(learned_kde.cpd(j), HCKDE) :
            sum = sum + 1
    sum_sum.append(sum/10*100)
print("Hill climbing with val-lik")
print("MSE:",round(np.mean(accuracy),2),"+-",round(np.std(accuracy),2))
print("LL:",round(np.mean(ll),2),"+-",round(np.std(ll),2))
print("Num arcs:",round(np.mean(num_arcs),2),"+-",round(np.std(num_arcs),2))
print("Class arcs:",round(np.mean(class_arcs),2),"+-",round(np.std(class_arcs),2))
print("KDE%:",round(np.mean(sum_sum),2),"+-",round(np.std(sum_sum),2))
print("Time:",round(np.mean(times),2),"+-",round(np.std(times),2))
analysis_hc = pd.DataFrame(data = np.vstack([accuracy,ll,num_arcs,class_arcs,sum_sum,times]).transpose(), columns = ["MSE", "logl", "n_arcs", "class_arcs","kdes", "ptime"])
print("--------")

sum_sum = []
accuracy = []
ll = []
num_arcs = []
class_arcs = []
times = []
for i in range(n_exps) :
    t0 = time.process_time()
    #learned_kde = hc(df_train, start = naive, operators = ["node_type","arcs"], score = "validated-lik", num_folds = 10)
    learned_kde = MMHC().estimate(hypot_test = test, operators = OperatorPool([ChangeNodeTypeSet(),ArcOperatorSet()]), score = ValidatedLikelihood(df_train), bn_type = SemiparametricBNType())
    learned_kde.fit(df_train)
    times.append(time.process_time()-t0)
    saved = predict_class(df_test, learned_kde)
    accuracy.append(np.sum((labels-saved["True"])**2) / len(df_test.index))
    ll.append(learned_kde.slogl(df_train)/500)
    num_arcs.append(learned_kde.num_arcs())
    class_arcs.append(len([j for j in learned_kde.arcs() if "class" in j]))
    sum = 0
    for j in learned_kde.nodes()[:-2] :    
        if isinstance(learned_kde.cpd(j), CKDE) or isinstance(learned_kde.cpd(j), HCKDE) :
            sum = sum + 1
    sum_sum.append(sum/10*100)
print("MMHC with val-lik")
print("MSE:",round(np.mean(accuracy),2),"+-",round(np.std(accuracy),2))
print("LL:",round(np.mean(ll),2),"+-",round(np.std(ll),2))
print("Num arcs:",round(np.mean(num_arcs),2),"+-",round(np.std(num_arcs),2))
print("Class arcs:",round(np.mean(class_arcs),2),"+-",round(np.std(class_arcs),2))
print("KDE%:",round(np.mean(sum_sum),2),"+-",round(np.std(sum_sum),2))
print("Time:",round(np.mean(times),2),"+-",round(np.std(times),2))
analysis_mmhc_val = pd.DataFrame(data = np.vstack([accuracy,ll,num_arcs,class_arcs,sum_sum,times]).transpose(), columns = ["MSE", "logl", "n_arcs", "class_arcs","kdes", "ptime"])
print("--------")

sum_sum = []
accuracy = []
ll = []
num_arcs = []
class_arcs = []
times = []
for i in range(n_exps) :
    t0 = time.process_time()
    #learned_kde = hc(df_train, start = naive, operators = ["node_type","arcs"], score = "validated-lik", num_folds = 10)
    learned_kde = MMHC().estimate(hypot_test = test, operators = OperatorPool([ChangeNodeTypeSet(),ArcOperatorSet()]), score = CVLikelihood(df_train), bn_type = SemiparametricBNType())
    learned_kde.fit(df_train)
    times.append(time.process_time()-t0)
    saved = predict_class(df_test, learned_kde)
    accuracy.append(np.sum((labels-saved["True"])**2) / len(df_test.index))
    ll.append(learned_kde.slogl(df_train)/500)
    num_arcs.append(learned_kde.num_arcs())
    class_arcs.append(len([j for j in learned_kde.arcs() if "class" in j]))
    sum = 0
    for j in learned_kde.nodes()[:-2] :    
        if isinstance(learned_kde.cpd(j), CKDE) or isinstance(learned_kde.cpd(j), HCKDE) :
            sum = sum + 1
    sum_sum.append(sum/10*100)
print("MMHC with cv-lik")
print("MSE:",round(np.mean(accuracy),2),"+-",round(np.std(accuracy),2))
print("LL:",round(np.mean(ll),2),"+-",round(np.std(ll),2))
print("Num arcs:",round(np.mean(num_arcs),2),"+-",round(np.std(num_arcs),2))
print("Class arcs:",round(np.mean(class_arcs),2),"+-",round(np.std(class_arcs),2))
print("KDE%:",round(np.mean(sum_sum),2),"+-",round(np.std(sum_sum),2))
print("Time:",round(np.mean(times),2),"+-",round(np.std(times),2))
analysis_mmhc_cv = pd.DataFrame(data = np.vstack([accuracy,ll,num_arcs,class_arcs,sum_sum,times]).transpose(), columns = ["MSE", "logl", "n_arcs", "class_arcs","kdes", "ptime"])
print("--------")

Hill climbing with val-lik
MSE: 0.16 +- 0.01
LL: -32.78 +- 4.03
Num arcs: 36.7 +- 6.91
Class arcs: 10.85 +- 0.48
KDE%: 88.5 +- 7.92
Time: 26.52 +- 10.74
--------
MMHC with val-lik
MSE: 0.19 +- 0.01
LL: -37.66 +- 1.87
Num arcs: 23.1 +- 4.09
Class arcs: 4.05 +- 1.86
KDE%: 93.5 +- 9.1
Time: 12.28 +- 2.72
--------
MMHC with cv-lik
MSE: 0.18 +- 0.0
LL: -35.87 +- 0.14
Num arcs: 28.85 +- 0.36
Class arcs: 7.0 +- 0.0
KDE%: 100.0 +- 0.0
Time: 19.91 +- 1.64
--------


In [75]:
analysis_ga_cv

,MSE,logl,n_arcs,class_arcs,kdes,ptime
0,0.193622,-46.211505,53.0,10.0,0.0,1.459491
1,0.192682,-46.204224,56.0,11.0,0.0,1.393674
2,0.193762,-46.185867,61.0,11.0,0.0,2.557138
3,0.194243,-46.203636,57.0,11.0,0.0,1.479339
4,0.193261,-46.211125,55.0,10.0,0.0,1.286369
5,0.191463,-46.189103,57.0,11.0,0.0,1.849517
6,0.192333,-46.206194,53.0,11.0,0.0,1.214592
7,0.193319,-46.189057,59.0,11.0,0.0,3.024686
8,0.194065,-46.192206,60.0,11.0,0.0,1.686018
9,0.191659,-46.193108,55.0,11.0,0.0,1.733318


In [72]:
n_exps = 20
df = df.sample(frac = 1)
df_train = df.head(2000)
for i in df_train.columns[:-2] :
    df_train = df_train[df_train[i] <4]
    df_train = df_train[df_train[i] >-4]
df_test = df.tail(500)

for i in df_test.columns[:-2] :
    df_test = df_test[df_test[i] <4]
    df_test = df_test[df_test[i] >-4]
    

    
labels = np.zeros(shape = (len(df_test.index)))
labels[df_test["class"] == "True"] = 1

sum_sum = []
accuracy = []
ll = []
num_arcs = []
class_arcs = []
times = []
for i in range(n_exps) :
    t0 = time.process_time()
    learned_kde = hc(df_train, bn_type = CLGNetworkType(), operators = ["arcs"], score = "bic", num_folds = 10)
    #MMHC().estimate(hypot_test = test, operators = OperatorPool([ChangeNodeTypeSet(),ArcOperatorSet()]), score = CVLikelihood(df_train), bn_type = SemiparametricBNType())
    learned_kde.fit(df_train)
    times.append(time.process_time()-t0)
    saved = predict_class(df_test, learned_kde)
    accuracy.append(np.sum((labels-saved["True"])**2) / len(df_test.index))
    ll.append(learned_kde.slogl(df_train)/500)
    num_arcs.append(learned_kde.num_arcs())
    class_arcs.append(len([j for j in learned_kde.arcs() if "class" in j]))
    sum = 0
    for j in learned_kde.nodes()[:-2] :    
        if isinstance(learned_kde.cpd(j), CKDE) or isinstance(learned_kde.cpd(j), HCKDE) :
            sum = sum + 1
    sum_sum.append(sum/10*100)
print("Hill climbing with BIC")
print("MSE:",round(np.mean(accuracy),2),"+-",round(np.std(accuracy),2))
print("LL:",round(np.mean(ll),2),"+-",round(np.std(ll),2))
print("Num arcs:",round(np.mean(num_arcs),2),"+-",round(np.std(num_arcs),2))
print("Class arcs:",round(np.mean(class_arcs),2),"+-",round(np.std(class_arcs),2))
print("KDE%:",round(np.mean(sum_sum),2),"+-",round(np.std(sum_sum),2))
print("Time:",round(np.mean(times),2),"+-",round(np.std(times),2))
analysis_ga_bic = pd.DataFrame(data = np.vstack([accuracy,ll,num_arcs,class_arcs,sum_sum,times]).transpose(), columns = ["MSE", "logl", "n_arcs", "class_arcs","kdes", "ptime"])
print("--------")


sum_sum = []
accuracy = []
ll = []
num_arcs = []
class_arcs = []
times = []
for i in range(n_exps) :
    t0 = time.process_time()
    learned_kde = hc(df_train, bn_type = CLGNetworkType(), operators = ["arcs"], score = "validated-lik", num_folds = 10)
    #MMHC().estimate(hypot_test = test, operators = OperatorPool([ChangeNodeTypeSet(),ArcOperatorSet()]), score = CVLikelihood(df_train), bn_type = SemiparametricBNType())
    learned_kde.fit(df_train)
    times.append(time.process_time()-t0)
    saved = predict_class(df_test, learned_kde)
    accuracy.append(np.sum((labels-saved["True"])**2) / len(df_test.index))
    ll.append(learned_kde.slogl(df_train)/500)
    num_arcs.append(learned_kde.num_arcs())
    class_arcs.append(len([j for j in learned_kde.arcs() if "class" in j]))
    sum = 0
    for j in learned_kde.nodes()[:-2] :    
        if isinstance(learned_kde.cpd(j), CKDE) or isinstance(learned_kde.cpd(j), HCKDE) :
            sum = sum + 1
    sum_sum.append(sum/10*100)
print("Hill climbing with val-lik")
print("MSE:",round(np.mean(accuracy),2),"+-",round(np.std(accuracy),2))
print("LL:",round(np.mean(ll),2),"+-",round(np.std(ll),2))
print("Num arcs:",round(np.mean(num_arcs),2),"+-",round(np.std(num_arcs),2))
print("Class arcs:",round(np.mean(class_arcs),2),"+-",round(np.std(class_arcs),2))
print("KDE%:",round(np.mean(sum_sum),2),"+-",round(np.std(sum_sum),2))
print("Time:",round(np.mean(times),2),"+-",round(np.std(times),2))
analysis_ga_val = pd.DataFrame(data = np.vstack([accuracy,ll,num_arcs,class_arcs,sum_sum,times]).transpose(), columns = ["MSE", "logl", "n_arcs", "class_arcs","kdes", "ptime"])
print("--------")


sum_sum = []
accuracy = []
ll = []
num_arcs = []
class_arcs = []
times = []
for i in range(n_exps) :
    t0 = time.process_time()
    learned_kde = hc(df_train, bn_type = CLGNetworkType(), operators = ["arcs"], score = "cv-lik", num_folds = 10)
    #MMHC().estimate(hypot_test = test, operators = OperatorPool([ChangeNodeTypeSet(),ArcOperatorSet()]), score = CVLikelihood(df_train), bn_type = SemiparametricBNType())
    learned_kde.fit(df_train)
    times.append(time.process_time()-t0)
    saved = predict_class(df_test, learned_kde)
    accuracy.append(np.sum((labels-saved["True"])**2) / len(df_test.index))
    ll.append(learned_kde.slogl(df_train)/500)
    num_arcs.append(learned_kde.num_arcs())
    class_arcs.append(len([j for j in learned_kde.arcs() if "class" in j]))
    sum = 0
    for j in learned_kde.nodes()[:-2] :    
        if isinstance(learned_kde.cpd(j), CKDE) or isinstance(learned_kde.cpd(j), HCKDE) :
            sum = sum + 1
    sum_sum.append(sum/10*100)
print("Hill climbing with cv-lik")
print("MSE:",round(np.mean(accuracy),2),"+-",round(np.std(accuracy),2))
print("LL:",round(np.mean(ll),2),"+-",round(np.std(ll),2))
print("Num arcs:",round(np.mean(num_arcs),2),"+-",round(np.std(num_arcs),2))
print("Class arcs:",round(np.mean(class_arcs),2),"+-",round(np.std(class_arcs),2))
print("KDE%:",round(np.mean(sum_sum),2),"+-",round(np.std(sum_sum),2))
print("Time:",round(np.mean(times),2),"+-",round(np.std(times),2))
analysis_ga_cv = pd.DataFrame(data = np.vstack([accuracy,ll,num_arcs,class_arcs,sum_sum,times]).transpose(), columns = ["MSE", "logl", "n_arcs", "class_arcs","kdes", "ptime"])
print("--------")

Hill climbing with BIC
MSE: 0.19 +- 0.0
LL: -46.32 +- 0.0
Num arcs: 50.0 +- 0.0
Class arcs: 9.0 +- 0.0
KDE%: 0.0 +- 0.0
Time: 0.06 +- 0.0
--------
Hill climbing with val-lik
MSE: 0.2 +- 0.0
LL: -47.1 +- 0.49
Num arcs: 32.7 +- 5.17
Class arcs: 6.0 +- 2.05
KDE%: 0.0 +- 0.0
Time: 0.44 +- 0.11
--------
Hill climbing with cv-lik
MSE: 0.19 +- 0.0
LL: -46.2 +- 0.01
Num arcs: 56.2 +- 2.6
Class arcs: 10.75 +- 0.43
KDE%: 0.0 +- 0.0
Time: 1.77 +- 0.54
--------


In [66]:
from scipy.stats import kstest
from scipy.stats import norm

for i in df.columns[:-2] : 
    print(i,kstest(df[i], 'norm'))


fixed acidity KstestResult(statistic=0.14433382926856342, pvalue=6.806213913198701e-47, statistic_location=0.1729773584075554, statistic_sign=1)
volatile acidity KstestResult(statistic=0.1427379704356445, pvalue=7.228441082937421e-46, statistic_location=0.061327369979953424, statistic_sign=1)
citric acid KstestResult(statistic=0.09933197373690558, pvalue=2.1751231446786863e-22, statistic_location=-0.6365354915635609, statistic_sign=-1)
residual sugar KstestResult(statistic=0.19657333596163662, pvalue=6.010921511702523e-87, statistic_location=-0.5145702936416315, statistic_sign=1)
chlorides KstestResult(statistic=0.1789337559025822, pvalue=5.181167920837191e-72, statistic_location=0.08746373870199499, statistic_sign=1)
free sulfur dioxide KstestResult(statistic=0.05764003476126494, pvalue=8.114116739755302e-08, statistic_location=-1.5715146471652022, statistic_sign=-1)
total sulfur dioxide KstestResult(statistic=0.04586395283077263, pvalue=4.1649165020146406e-05, statistic_location=-1.1

In [52]:
df2 =df2.drop("index",axis=1)

In [35]:
df2 = df.copy()
for i in df2.columns[:-2] :
    df2 = df2[df2[i] <3]
    df2 = df2[df2[i] >-3]

In [36]:
len(df2.index)/len(df.index)

0.9197337509788567

In [372]:
sum = 0
for i in learned_kde.nodes()[:-2] :    
    if isinstance(learned_kde.cpd(i), CKDE) or isinstance(learned_kde.cpd(i), HCKDE) :
        sum = sum + 1
print(sum/10 * 100)

100.0


In [356]:
me_hc_h = 0.564
me_mmhc_h = 0.52
me_hc_cv = 0.59
me_mmhc_cv = 0.46

0.4489779380314274

In [394]:
len([i for i in learned_kde.arcs() if "class" in i])

[('class', 'citric acid'), ('class', 'alcohol')]

In [54]:
accuracy(instance.drop("class", axis = 1), "True", learned_kde)/

array([0.5])

In [60]:
instance = df.iloc[[0]]
instance["class"] = pd.Categorical(["False"], categories=learned_kde.cpd("class").variable_values())
(math.e ** learned.logl(instance)) / likelihood(instance.drop("class", axis = 1), learned_kde)

/tmp/ipykernel_22436/3126163979.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  instance["class"] = pd.Categorical(["False"], categories=learned_kde.cpd("class").variable_values())


array([0.03584753])

In [ ]:
n_vertex = 0
x_og = df.iloc[[0]]
bayesace = BayesACE(bayesian_network=learned_kde, features= df.columns[:-1], n_vertex=n_vertex, chunks = 100, penalty=1, pop_size=50, likelihood_threshold=0.0, accuracy_threshold= 0.5, generations=10)
res = bayesace.run(x_og)

In [ ]:
res.counterfactual

In [ ]:
df.iloc[0]

In [ ]:
x_1 = x_og.drop("class",axis = 1)
df_vertex = res.path
to_plot = df.drop("class", axis = 1)
colours = df["class"].to_numpy()
colours[colours == "a"] = "green"
colours[colours == "b"] = "yellow"
colours[colours == "c"] = "blue"
plt.scatter(to_plot["x"],to_plot["y"], color = colours)
plt.plot(df_vertex.x,df_vertex.y,color = "red")
plt.show()

In [ ]:
n_vertex = 1
x_og = df.iloc[[0]]
face = FACE(bayesian_network=learned_kde, features= df.columns[:-1], dataset=df.drop("class", axis = 1), graph_type = "epsilon", chunks = 10, penalty=1, distance_threshold=0.75, accuracy_threshold= 0.5, verbose = False)
res = face.run(x_og)

In [ ]:
x_1 = x_og.drop("class",axis = 1)
df_vertex = res.path
to_plot = df.drop("class", axis = 1)
colours = df["class"].to_numpy()
colours[colours == "a"] = "green"
colours[colours == "b"] = "yellow"
colours[colours == "c"] = "blue"
plt.scatter(to_plot["x"],to_plot["y"], color = colours)
plt.plot(df_vertex.x,df_vertex.y,color = "red")
plt.show()